**ids-pdl10-tut.ipynb**: This Jupyter notebook is provided by Joachim Vogt for the *Python Data Lab* of the module *CH-700 Introduction to Data Science* offered in Fall 2023 at Constructor University. Jupyter notebooks and other learning resources are available from a dedicated *module platform*.

# First steps with scikit-learn

This tutorial discusses a curve fitting example in two contexts, namely, from a traditional parametric data modeling perspective as well as an application of the Machine Learning (ML) methodology implemented in the Python package scikit-learn. Follow the instructions below to learn to

- [ ] understand the basics of least squares parametric data modeling,
- [ ] apply the function `polyfit()` from the NumPy module,
- [ ] work with NumPy's `Polynomial` class,
- [ ] understand the logic and the interface of the ML package scikit-learn,
- [ ] apply the model class `LinearRegression`from `sklearn.linear_model`,
- [ ] distinguish between training scores and validation scores in model evaluation.

If you wish to keep track of your progress, you may edit this markdown cell, check a box in the list above after having worked through the respective part of this notebook, and save the file.

*Short exercises* are embedded in this notebook. *Sample solutions* can be found at the end of the document.

## Preparation

The following data file is expected to reside in the working directory. Identify the file on the module platform and upload it to the same folder as this Jupyter notebook.

- `co2_mm_mlo.txt`: [Mauna Loa CO$_2$ monthly mean data](https://gml.noaa.gov/webdata/ccgg/trends/co2/co2_mm_mlo.txt) hosted by [NOAA's Global Monitoring Laboratory](https://gml.noaa.gov/).

Run the following code cell to import standard Python data science libraries. The NumPy module facilitates efficient processing of numerical arrays, and is usually imported as `np`. From the matplotlib library we import the package `pyplot` using the standard abbreviation `plt`. The magic command `%matplotlib inline` (IPython shell) allows for inline display of graphics.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Importing scikit-learn

The Python package scikit-learn is imported as `sklearn`. Dedicated modules providing model classes (estimator objects) are imported when needed, see below.

### scikit-learn documentation and tutorials

The scikit-learn package is hosted at [scikit-learn.org](https://scikit-learn.org/stable/index.html), with an extensive [user guide](https://scikit-learn.org/stable/user_guide.html) and [tutorials](https://scikit-learn.org/stable/tutorial/index.html).

## Parametric modeling approach to curve fitting

The section summarizes basic terminology in *parametric data modeling* with applications, e.g., in curve fitting. The basic setup considered here assumes that the observed *data $d$* result from a deterministic *model $m$* and an additive *random component $r$* (residual, error, noise). Data and model are ordered by an independent variable denoted as $t$, thus we write $d_t$, $d(t)$, $m_t$, etc.

- Model equations: $d_t = m_t + r_t$ or $d (t) = m (t) + r (t)$.
- To make model parameters (vector $\mathbf{q}$) explicit, we write $m = m(t|\mathbf{q})$.
- When $d_t$, $m_t$, $r_t$ are interpreted as components of vectors $\mathbf{d}$, $\mathbf{m}$, $\mathbf{r}$, the vectorial form of the parametric model reads $\mathbf{d} \, = \, \mathbf{m} + \mathbf{r}$.

### Least squares approach to parametric data modeling

In the *least squares approach to parametric data modeling*, the optimum model is found by minimizing the error-weighted square deviation of data and model.
$$
\chi^2 
\; = \; \sum_t \left( \frac{d (t) - m(t|\mathbf{q})}{\sigma(t)} \right)^2 
\; = \; \sum_t \left( \frac{r(t|\mathbf{q})}{\sigma(t)} \right)^2 
\; \stackrel{!}{=} \; \text{Min} ~.
$$
Error scaling: measurement errors $\sigma (t)$ can be absorbed in $d(t)$ and $m(t)$ by replacing $m(t) \rightarrow m (t) / \sigma(t)$ and $d(t) \rightarrow d(t) / \sigma(t)$, then 
$$
\chi^2 \, \propto \, |\mathbf{r} |^2 \, = \, |\mathbf{d} - \mathbf{m} |^2 
\; \stackrel{!}{=} \; \text{Min} ~.
$$

If all parameters $a_0,a_1,a_2,\ldots$ enter the model function $m = m(t) = m(t|\mathbf{a})$ 
linearly, they are called *amplitudes* or *coefficients*, and the model function can be written in the form
$$
m (t) \; = \; \sum_k a_k f_k (t) ~.
$$
The given functions $f_k = f_k(t)$, $k=0,1,2,3,\ldots$, are called *basis functions*.

The task of fitting a polynomial to a set of measurements can be addressed by selecting as basis functions integer powers of the independent variable $t$: $f_k (t) = t^k, k = 0,1,2,...$ The model function is given by
$$
m(t) 
\; = \; a_0 \, + \, a_1  \cdot t \, + \, a_2 \cdot t^2 \, + \, \cdots 
\; = \; \sum_k a_k \cdot t^k ~.
$$
The independent variable $t$ can be interpreted as time.

### NumPy function `polyfit()`

Least squares polynomial fitting is implemented in several Python libraries. To demonstrate the procedure in a basic manner, we employ the NumPy function `polyfit()`. The `fit()` method of the NumPy `Polynomial` class, recommended for reasons of numerical stability, is applied further below in an exercise. 

We start by creating a synthetic data set.

In [ ]:
a0 = 1                                      #.. y-intercept of the straight line
a1 = 3                                      #.. slope of the straight line
namp = 4                                    #.. noise amplitude
time = np.linspace(0,5,21)                  #.. independent variable
gmod = a0 + a1*time                         #.. generating linear model
rng = np.random.default_rng(99)             #.. initialize random number generator
noise = namp*rng.standard_normal(time.size) #.. noise (error) term
data = gmod + noise                         #.. synthetic measurements
plt.plot(time,gmod,color='green',label='Generating model')
plt.scatter(time,data,color='blue',label='Noisy measurements')
plt.title('Straight-line fit: Synthetic data set')
plt.legend()
plt.grid()

The NumPy function `polyfit()` returns the estimated polynomial coefficients in reverse order.

In [ ]:
degree = 1
mcoef = np.polyfit(time,data,degree) #.. obtain vector of estimated model coefficients
print('Estimated y-intercept : ',mcoef[1])
print('Estimated slope       : ',mcoef[0])

Using the estimated coefficients, the model is evaluated on the array of independent variables to yield the predictions.

In [ ]:
mpred = np.polyval(mcoef,time)        #.. compute model predictions
plt.plot(time,gmod,color='green',label='Generating model')
plt.scatter(time,data,color='blue',label='Noisy measurements')
plt.plot(time,mpred,color='red',label='Model predictions')
plt.title('Straight-line fit: Synthetic data set and estimated model')
plt.legend()
plt.grid()

Despite the large scatter in the measurements, the straight line fit is quite close to the generating linear model.

## Polynomial fit of the Mauna Loa CO$_2$ time series

The file `co2_mm_mlo.txt` as obtained from [NOAA's Global Monitoring Laboratory](https://gml.noaa.gov/) contains monthly mean atmospheric CO$_2$ concentrations measured at the Mauna Loa observatory. Click on the filename in the directory listing to display the content of this plain text file. Note the contextual information is in a header separated from the data stored in several columns.   

The code in the following cell loads two columns from the Mauna Loa CO$_2$ data file: fractional year and monthly average CO$_2$ concentrations. The data are displayed as a time series.

In [ ]:
### From the data file load columns (2,3) and store in two vectors.
time,data = np.genfromtxt('co2_mm_mlo.txt',usecols=(2,3),unpack=True)
plt.figure(figsize=(9,4))
plt.plot(time,data,linewidth=2,color='red')
plt.title('Mauna Loa CO$_2$ time series')
plt.xlabel('Year')
plt.ylabel('CO$_2$ [ppm]')
plt.grid()

Inspection of the graph suggests that the time series can be modeled as a low-order polynomial, with annual variations of about $\pm 2$ppm and possibly random fluctuations treated as the residual. First, we try out a linear function (polynomial degree 1).

In [ ]:
degree = 1
mcoef = np.polyfit(time,data,degree)
print('Estimated y-intercept : ',mcoef[1])
print('Estimated slope       : ',mcoef[0])

To check if the model predictions vary from the measurements within the expected range of $\pm 2\,$ppm, we construct the residuals and compute their average amplitude in the rms (root-mean-square) sense, i.e., the rms deviation of the data from the model. The value may also be computed by calling the NumPy function `std()` (standard deviation).

In [ ]:
mpred = np.polyval(mcoef,time)
resid = data-mpred
print('rms deviation of data and model: ',np.sqrt(np.mean(resid**2)))

This value is larger than expected, indicating that we have not yet identified the appropriate model. We thus continue with a quadratic fit of the time series.

In [ ]:
degree = 2
mcoef = np.polyfit(time,data,degree)
mpred = np.polyval(mcoef,time)
resid = data-mpred
print('rms deviation of data and model: ',np.sqrt(np.mean(resid**2)))

This value is consistent with the expected residual amplitude of about $\pm 2$ppm, indicating that we have identified a suitable model. To check if the residual improves further, we try out also higher-order polynomials up to degree 5. 

In [ ]:
for degree in range(6):
    mcoef = np.polyfit(time,data,degree)
    mpred = np.polyval(mcoef,time)
    resid = data-mpred
    print('degree: {deg:.0f}, rms deviation: {rms:.4f}'.format(
        deg=degree,rms=np.sqrt(np.mean(resid**2))))

The average size of the residuals does not improve signficantly if degrees larger than 2 are used in this parametric modeling exercise. To further corroborate a quadratic as an appropriate choice for the model function, we plot the data together with the linear and the quadratic models as well as histograms of the corresponding residuals. The distribution of quadratic residuals is clearly narrower than the distribution of linear residuals.

In [ ]:
### Compute linear and quadratic model predictions.
mcoef1 = np.polyfit(time,data,1)
mpred1 = np.polyval(mcoef1,time)
resid1 = data-mpred1
mcoef2 = np.polyfit(time,data,2)
mpred2 = np.polyval(mcoef2,time)
resid2 = data-mpred2
plt.figure(figsize=(11,4))
### Plot measured CO2 time series and polynomial fits.
plt.subplot(121)
plt.plot(time,data,linewidth=2,color='red',label='Measurements')
plt.plot(time,mpred1,linewidth=2,color='green',label='Linear fit')
plt.plot(time,mpred2,linewidth=2,color='blue',label='Quadratic fit')
plt.title(r'Mauna Loa CO$_2$ time series')
plt.xlabel(r'Year')
plt.ylabel(r'CO$_2$ [ppm]')
plt.legend()
plt.grid()
### Compute residuals and plot histograms.
plt.subplot(122)
bin_edges = np.linspace(-15,15,31)
plt.hist(np.stack([resid1,resid2],axis=1),bins=bin_edges,color=['green','blue'],
         label=[r'Linear CO$_2$ residuals',r'Quadratic CO$_2$ residuals'])
plt.title(r'Histograms of CO$_2$ residuals')
plt.xlabel('CO$_2$ [ppm]')
plt.ylabel('Occurence frequency')
plt.xlim([-15,15])
plt.grid()
plt.legend(loc=3)

### Exercise: NumPy `Polynomial` class

In NumPy, polynomial fitting is implemented through the `fit()` method as part of the `Polynomial` (sub-)package, imported below using the prefix `Poly`. Consult the NumPy documentation to familiarize with the logic and the methods associated with this package.

In [ ]:
from numpy.polynomial import Polynomial as Poly

Repeat the following steps in the Mauna Loa CO$_2$ polynomial fit example but using the `fit()` method of NumPy's `Polynomial` class instead of the function `polyfit()`:

- After loading the data, compute and display the rms deviations for polynomials up to degree 5.
- Plot the measurements together with the linear and quadratic fits, and display the histograms of the resulting residuals.

In [ ]:
### RMS deviations for polynomials up to degree 5.

### Compute linear and quadratic model predictions.

### Plot measured CO2 time series and polynomial fits.

### Compute residuals and plot histograms.


## Machine Learning (ML) approach to curve fitting

The polynomial fit example illustrates typical steps and choices in a modeling exercise. After 

- *the model class is selected* (such as the polynomial class of model functions) and 
- *the data are provided in an appropriate format*,
- *the model is fitted to the data* (i.e., a particular instance of the model class is found that yields is closest to the obserations, such as a quadratic with a specific set of coefficients),
- *the model predictions are computed*, and
- *the model is validated* (i.e., the choices made in the modeling process are double-checked using a suitable metric such as the rms deviation in the polynomial fitting example above).

To serve as a guideline for the general Machine Learning (ML) approach adopted in the Python package scikit-learn (available as `sklearn`) this list does not make explicit a central concept of parametric data modeling, namely, the set of model parameters. To make the ML approach more concrete, the Mauna Loa CO$_2$ time series fit example is revisited using the ML approach.

### Select the model class

To fit a model function that is linear in the parameters, the model class `LinearRegression` from `sklearn.linear_model` can be employed. More technically, such model classes are referred to as estimator objects. The keyword argument `fit_intercept` controls if a constant offset is supposed to be considered separately. For didactic reasons, we abstain from activating this option and instead incorporate the corresponding basis function explicitly.

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=False)

### Provide the data in an appropriate format

The ML package Scikit-learn expects the data in a specific format. The variable to be modeled needs to be stored in a so-called *target vector*, i.e., a one-dimensional array with its length denoted as `n_samples`. Setting the keyword `unpack` to `True`, this is accomplished by the NumPy function `genfromtxt()`. The target vector is stored in the array `y`. 

In [ ]:
time,y = np.genfromtxt('co2_mm_mlo.txt',usecols=(2,3),unpack=True)
print('Shape of array y : ',y.shape)

In the terminology of the regression example considered here, the basis functions are to be arranged in a so-called *features matrix* of shape `(n_samples,n_features)`. If we intend to fit a linear function $m(t) = a_0 + a_1 t$, column 0 contains an array of ones (basis function associated with the coefficient $a_0$) and column 1 contains the time array (basis function associated with the coefficient $a_1$). The features matrix is stored in the array `X`.

In [ ]:
X = np.stack([np.ones(time.size),time],axis=1)
print('Shape of array X : ',X.shape)

### Fit the model to the data

A scikit-learn model class is generically equipped with a `fit()` method. When called with the features matrix and the target vector, an instance of the model class is chosen that provides the best fit. If required, the model coefficients can be accessed through the `coef_` attribute.

In [ ]:
model.fit(X,y)
print('Estimated y-intercept : ',model.coef_[0])
print('Estimated slope       : ',model.coef_[1])

### Compute model predictions

The `predict()` method provides the model predictions.

In [ ]:
yprd = model.predict(X)

The predictions are plotted with the data in the same way as before.

In [ ]:
plt.plot(time,y,linewidth=2,color='red',label='Measurements')
plt.plot(time,yprd,linewidth=2,color='green',label='Linear fit')
plt.title(r'Mauna Loa CO$_2$ time series')
plt.xlabel(r'Year')
plt.ylabel(r'CO$_2$ [ppm]')
plt.legend()
plt.grid()

The scikit-learn interface provides the `score()` method to measure the fit quality. The output is usually in the range between 0 and 1, with better fits indicated by larger values. For the linear regression model discussed here, `score()` gives the coefficient of determination relative to a simple reference predictor, namely, the mean of the target values. If the model performs worse than the simple reference predictor, the value returned by `score()` can be negative.

In [ ]:
model.score(X,y)

While a score so close to 1 is encouraging, it does not serve as a proper model validation metric because the model was evaluated with the same data as used for training. A more meaningful approach to model validation makes use of holdout sets, see below.

### Exercise: Quadratic fit using scikit-learn

Using the model class `LinearRegression` from `sklearn.linear_model`, produce a quadratic fit of the Mauna Loa CO$_2$ time series.

In [ ]:
### Select the model class.

### Provide the data in an appropriate format.

### Fit the model to the data.

### Compute model predictions.

### Plot data and model predictions.


## Training and validation scores

In ML terminology, fitting a model, i.e., finding the most suitable representative of a chosen model class is often referred to as *model training*. Checking how well this particular representative performs in general applications is called *model testing* or *model validation*.

### Model validation using holdout sets

A popular model validation strategy uses only one part of the available data for training and keeps the other part, the so-called holdout set(s), for testing. In scikit-learn, this kind of model validation can be easily accomplished as follows. After importing `train_test_split`, we split the features matrix and the target vector with 50% in both the training and the validation data set.

In [ ]:
from sklearn.model_selection import train_test_split
X_trn,X_val,y_trn,y_val = train_test_split(X,y,train_size=0.5,random_state=99)

The training data are now used to fit the model to the data. Finally, both the training score and the validation score are computed. The values turn out to be quite close, indicating that the chosen model is at least not too complex for the data set. To properly interpret training scores versus validation scores, consult  [section 3. Model selection and evaluation](https://scikit-learn.org/stable/model_selection.html) of the [scikit-learn user guide](https://scikit-learn.org/stable/user_guide.html).

In [ ]:
model.fit(X_trn,y_trn)
print('Training score   : ',model.score(X_trn,y_trn))
print('Validation score : ',model.score(X_val,y_val))

### Exercise: Cross-validation

A model validation technique generalizing the holdout set approach is *cross-validation*. Here the data are splitted several times into pairs of sets for training and testing. Consult the scikit-learn documentation to understand how cross-validation is implemented, then write a small piece of code to apply cross-validation to the Mauna Loa CO$_2$ time series example.

In [ ]:
### Select the model class.

### Provide the data in an appropriate format.

### Cross-validation.


---
---

## Solutions

### Solution: NumPy `Polynomial` class

In [ ]:
from numpy.polynomial import Polynomial as Poly
time,data = np.genfromtxt('co2_mm_mlo.txt',usecols=(2,3),unpack=True)
### RMS deviations for polynomials up to degree 5.
for degree in range(6):
    model = Poly.fit(time,data,degree)
    resid = data-model(time)
    print('degree: {deg:.0f}, rms deviation: {rms:.4f}'.format(
        deg=degree,rms=np.sqrt(np.mean(resid**2))))
### Compute linear and quadratic model predictions.
model1 = Poly.fit(time,data,1)
mpred1 = model1(time)
model2 = Poly.fit(time,data,2)
mpred2 = model2(time)
plt.figure(figsize=(11,4))
### Plot measured CO2 time series and polynomial fits.
plt.subplot(121)
plt.plot(time,data,linewidth=2,color='red',label='Measurements')
plt.plot(time,mpred1,linewidth=2,color='green',label='Linear fit')
plt.plot(time,mpred2,linewidth=2,color='blue',label='Quadratic fit')
plt.title(r'Mauna Loa CO$_2$ time series')
plt.xlabel(r'Year')
plt.ylabel(r'CO$_2$ [ppm]')
plt.legend()
plt.grid()
### Compute residuals and plot histograms.
resid1 = data-mpred1
resid2 = data-mpred2
plt.subplot(122)
bin_edges = np.linspace(-15,15,31)
plt.hist(np.stack([resid1,resid2],axis=1),bins=bin_edges,color=['green','blue'],
         label=[r'Linear CO$_2$ residuals',r'Quadratic CO$_2$ residuals'])
plt.title(r'Histograms of CO$_2$ residuals')
plt.xlabel('CO$_2$ [ppm]')
plt.ylabel('Occurence frequency')
plt.xlim([-15,15])
plt.grid()
plt.legend(loc=3)

### Solution: Quadratic fit using scikit-learn

In [ ]:
### Select the model class.
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=False)
### Provide the data in an appropriate format.
time,y = np.genfromtxt('co2_mm_mlo.txt',usecols=(2,3),unpack=True)
X = np.stack([np.ones(time.size),time,time**2],axis=1)
print('Shape of array X : ',X.shape)
### Fit the model to the data.
model.fit(X,y)
### Compute model predictions.
yprd = model.predict(X)
### Plot data and model predictions.
plt.plot(time,y,linewidth=2,color='red',label='Measurements')
plt.plot(time,yprd,linewidth=2,color='blue',label='Quadratic fit')
plt.title(r'Mauna Loa CO$_2$ time series')
plt.xlabel(r'Year')
plt.ylabel(r'CO$_2$ [ppm]')
plt.legend()
plt.grid()

### Solution: Cross-validation

In [ ]:
### Select the model class.
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=False)
### Provide the data in an appropriate format.
time,y = np.genfromtxt('co2_mm_mlo.txt',usecols=(2,3),unpack=True)
X = np.stack([np.ones(time.size),time,time**2],axis=1) #.. quadratic fit
### Cross-validation.
from sklearn.model_selection import cross_val_score
cross_val_score(model,X,y,cv=3)

---
---